In [ ]:
%%capture
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 0
%config SqlMagic.displaycon = False

In [ ]:
import datetime

start = datetime.datetime.now() - datetime.timedelta(days=90)
end = datetime.datetime.now()

In [ ]:
%%sql cities_with_repeat_incident_locations <<

SELECT location_city_town, location_name, COUNT(*) count
FROM cts_forms_report
WHERE location_name notnull
GROUP BY location_city_town, location_name
HAVING COUNT(*) > 1
ORDER BY count desc
LIMIT 20

In [ ]:
def get_report_location_names(location_city='(any)'):
    results = %sql SELECT location_name, \
            COUNT(*) count \
            FROM cts_forms_report \
            WHERE location_city_town = '{{ location_city }}' \
            GROUP BY location_name \
            HAVING COUNT(*) > 1 \
            ORDER BY count desc \

    return results.DataFrame()

In [ ]:
from IPython.display import display, HTML
import numpy as np

readable_start = start.strftime("%B %d, %Y")
readable_end = end.strftime("%B %d, %Y")
city_data = cities_with_repeat_incident_locations.DataFrame()
cities = city_data[city_data['location_city_town'].notnull()]['location_city_town'].values
location_city_divs = []
location_city_dropdown = []
for city in np.unique(cities):
    location_city_dropdown.append(
        f'<option value="{city}">{city}</option>'
    )
    results = get_report_location_names(city)
    location_names = results['location_name'].values
    location_name_divs = []
    for location_name in location_names:
        count = results[results['location_name'] == location_name]['count'].values[0]
        location_name_divs.append(
            f'<div class="flex-row flex-justify display-flex location-{location_name}"><span>{location_name}</span> <strong>{count}</strong></div><hr/>'
        )
    location_name_divs = ''.join(location_name_divs)
    location_city_divs.append(
        f'<div class="flex-row flex-justify display-flex city-wrapper {city}"><span>{location_name_divs}</span></div>'
    )

display(HTML(f"""
    <div class="crt-portal-card light">
    <h3>Location names and report count</h3>
    <select name="incident_location_city" class="usa-select incident-location-city" aria-label="Incident location city">
        <option value="">Select a city</option>
        {''.join(location_city_dropdown)}
    </select>
    <p style="font-size: 0.75rem">
      <em>Repeated report locations submitted between
        <strong>{readable_start}</strong> and
        <strong>{readable_end}</strong>
      </em>
    </p>
    <br />
 
    {''.join(location_city_divs)}
    </div>
"""))